# Notebooks para preparar los datos para entrenar al modelo.

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('../database.db')
cursor = conn.cursor()

In [3]:
df_contagios = pd.read_sql_query('SELECT * FROM contagios', con=conn)
df_contagios.head()

,id_uta,departamento,provincia,ano,semanas_epidemiologicas,grupo_edad_id,grupo_edad_desc,cantidad_casos,poblacion,id_provincia
0,2003,COMUNA 3,BUENOS AIRES,2018,15.0,6,DE 15 A 19 ANOS,1.0,192945,0
1,2004,COMUNA 4,BUENOS AIRES,2018,6.0,6,DE 15 A 19 ANOS,1.0,239279,0
2,6091,BERAZATEGUI,BUENOS AIRES,2018,11.0,7,DE 20 A 24 ANOS,1.0,358262,0
3,6091,BERAZATEGUI,BUENOS AIRES,2018,14.0,9,DE 35 A 44 ANOS,2.0,358262,0
4,6091,BERAZATEGUI,BUENOS AIRES,2018,19.0,11,MAYORES DE 65 ANOS,1.0,358262,0


In [14]:
orden_dengue = ['provincia', 'departamento', 'ano', 'semanas_epidemiologicas']
df_contagios.groupby(
    by=orden_dengue
).sum().sort_values(orden_dengue)

id_uta  \
provincia    departamento           ano  semanas_epidemiologicas           
BUENOS AIRES ADOLFO GONZALES CHAVES 2023 16.0                       6014   
                                         29.0                       6014   
                                    2024 11.0                       6014   
                                         14.0                       6014   
             ALBERTI                2023 15.0                       6021   
...                                                                  ...   
TUCUMÁN      YERBA BUENA            2024 20.0                     901190   
                                         21.0                     630833   
                                         22.0                     360476   
                                         23.0                      90119   
                                         24.0                      90119   

                                                                  grupo_edad_id  \
provincia    departamento           ano  semanas_epidemiologicas                  
BUENOS AIRES ADOLFO GONZALES CHAVES 2023 16.0                                10   
                                         29.0                                 9   
                                    2024 11.0                                 8   
                                         14.0                                10   
             ALBERTI                2023 15.0                                10   
...                                                                         ...   
TUCUMÁN      YERBA BUENA            2024 20.0                                74   
                                         21.0                                60   
                                         22.0                                32   
                                         23.0                                 9   
                                         24.0                                10   

                                                                                                    grupo_edad_desc  \
provincia    departamento           ano  semanas_epidemiologicas                                                      
BUENOS AIRES ADOLFO GONZALES CHAVES 2023 16.0                                                       DE 45 A 65 ANOS   
                                         29.0                                                       DE 35 A 44 ANOS   
                                    2024 11.0                                                       DE 25 A 34 ANOS   
                                         14.0                                                       DE 45 A 65 ANOS   
             ALBERTI                2023 15.0                                                       DE 45 A 65 ANOS   
...                                                                                                             ...   
TUCUMÁN      YERBA BUENA            2024 20.0                     DE 45 A 65 ANOSDE 15 A 19 ANOSDE 20 A 24 ANOSD...   
                                         21.0                     DE 15 A 19 ANOSDE 45 A 65 ANOSMAYORES DE 65 AN...   
                                         22.0                     DE 10 A 14 ANOSDE 45 A 65 ANOSDE 35 A 44 ANOSD...   
                                         23.0                                                       DE 35 A 44 ANOS   
                                         24.0                                                       DE 45 A 65 ANOS   

                                                                  cantidad_casos  \
provincia    departamento           ano  semanas_epidemiologicas                   
BUENOS AIRES ADOLFO GONZALES CHAVES 2023 16.0                                1.0   
                                         29.0                                1.0   
                                    2024 11.0                                1.0   
                             

In [53]:
columnas_groupby = ['departamento', 'ano', 'semanas_epidemiologicas']
df_contagios_agg = df_contagios.groupby(columnas_groupby).agg({
    'cantidad_casos': 'sum',
    'poblacion': 'first',
    'id_provincia': 'first',
    'id_uta': 'first'
    }
).reset_index()

In [54]:
df_contagios_agg

,departamento,ano,semanas_epidemiologicas,cantidad_casos,poblacion,id_provincia,id_uta
0,1 DE MAYO,2020,8.0,2.0,12515,3,22126
1,1 DE MAYO,2020,9.0,7.0,12515,3,22126
2,1 DE MAYO,2020,10.0,4.0,12515,3,22126
3,1 DE MAYO,2020,11.0,10.0,12515,3,22126
4,1 DE MAYO,2020,12.0,4.0,12515,3,22126
...,...,...,...,...,...,...,...
15401,ZARATE,2024,19.0,2.0,132848,0,6882
15402,ZARATE,2024,25.0,1.0,132848,0,6882
15403,ZARATE,2025,17.0,1.0,133993,0,6882
15404,ZONDA,2024,12.0,2.0,6500,17,70133


In [77]:
contagios = pd.read_csv('../data/departamentos_con_estacion.csv')

In [92]:
df_clima_agg_full

,id_estacion,anio,semana,fecha,precipitacion_pluviometrica,temperatura_minima,temperatura_maxima,temperatura_media,humedad_media,rocio_medio,...,precipitacion_pluviometrica_lag3,temperatura_minima_lag3,temperatura_maxima_lag3,temperatura_media_lag3,humedad_media_lag3,rocio_medio_lag3,tesion_vapor_media_lag3,radiacion_global_lag3,heliofania_efectiva_lag3,heliofania_relativa_lag3
0,A872801,2018,5,2018-02-01 00:00:00,0.000000,19.814286,33.428571,26.380657,48.857143,16.066914,...,0.071282,20.857143,32.471429,26.344840,58.142857,17.314917,20.053073,19.498533,7.859546,55.142857
1,A872801,2018,6,2018-02-08 00:00:00,0.190602,20.000000,33.357143,26.508136,51.571429,16.009973,...,0.000000,17.800000,32.828571,25.153671,56.428571,16.678904,19.247406,23.093349,8.924465,63.428571
2,A872801,2018,7,2018-02-15 00:00:00,0.104044,16.542857,31.500000,23.878969,48.285714,13.606205,...,0.065213,18.557143,31.300000,24.316767,58.000000,16.353461,19.074726,22.105954,8.489425,61.142857
3,A872801,2018,1,2018-02-18 03:00:00,0.073696,14.925000,30.975000,23.480816,50.500000,12.928104,...,0.000000,19.814286,33.428571,26.380657,48.857143,16.066914,18.700631,24.891456,10.331836,75.285714
4,A872801,2018,8,2018-02-22 00:00:00,0.046151,16.657143,29.271429,22.894744,63.000000,15.696834,...,0.190602,20.000000,33.357143,26.508136,51.571429,16.009973,18.730467,21.886077,8.773545,65.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16265,NH0550,2025,33,2025-08-14 00:00:00,0.000000,7.042857,24.578571,15.639286,57.000000,8.708845,...,0.148167,9.714286,17.900000,13.557143,79.628571,8.376279,12.810905,5.980414,1.764286,16.658571
16266,NH0550,2025,34,2025-08-21 00:00:00,0.000000,6.371429,26.150000,15.714286,53.571429,8.432129,...,0.000000,8.164286,21.471429,15.207143,70.871429,9.492939,12.999786,9.007875,4.100000,38.070000
16267,NH0550,2025,35,2025-08-28 00:00:00,0.026685,4.814286,27.871429,16.764286,43.000000,8.629537,...,0.000000,4.742857,22.000000,13.371429,68.714286,6.997202,10.742250,12.402038,6.571429,60.180000
16268,NH0550,2025,36,2025-09-04 00:00:00,0.020187,4.542857,22.800000,13.671428,54.809524,7.155052,...,0.000000,7.042857,24.578571,15.639286,57.000000,8.708845,10.412464,12.996384,6.671429,60.365714


In [90]:
departamentos.sample()

,id_departamento,departamento,latitud,longitud,id_provincia,id_localidad
418,18133,SAN COSME,-27.363933,-58.661982,3,34


In [91]:
estaciones.sample()

,id_estacion,id_interno,nombre,latitud,longitud,id_provincia,id_localidad,id_departamento
140,676,A872941,TUPUNGATO - EEA LA CONSULTA,-33.03,-69.28,12,94,50063


In [96]:
merged = df_clima_agg_full.merge(
    estaciones[['id_estacion', 'id_interno', 'id_departamento']],
    how='inner',
    left_on='id_estacion',
    right_on='id_interno'
)

merged.head()

,id_estacion_x,anio,semana,fecha,precipitacion_pluviometrica,temperatura_minima,temperatura_maxima,temperatura_media,humedad_media,rocio_medio,...,temperatura_media_lag3,humedad_media_lag3,rocio_medio_lag3,tesion_vapor_media_lag3,radiacion_global_lag3,heliofania_efectiva_lag3,heliofania_relativa_lag3,id_estacion_y,id_interno,id_departamento
0,A872801,2018,5,2018-02-01 00:00:00,0.000000,19.814286,33.428571,26.380657,48.857143,16.066914,...,26.344840,58.142857,17.314917,20.053073,19.498533,7.859546,55.142857,79,A872801,6539
1,A872801,2018,6,2018-02-08 00:00:00,0.190602,20.000000,33.357143,26.508136,51.571429,16.009973,...,25.153671,56.428571,16.678904,19.247406,23.093349,8.924465,63.428571,79,A872801,6539
2,A872801,2018,7,2018-02-15 00:00:00,0.104044,16.542857,31.500000,23.878969,48.285714,13.606205,...,24.316767,58.000000,16.353461,19.074726,22.105954,8.489425,61.142857,79,A872801,6539
3,A872801,2018,1,2018-02-18 03:00:00,0.073696,14.925000,30.975000,23.480816,50.500000,12.928104,...,26.380657,48.857143,16.066914,18.700631,24.891456,10.331836,75.285714,79,A872801,6539
4,A872801,2018,8,2018-02-22 00:00:00,0.046151,16.657143,29.271429,22.894744,63.000000,15.696834,...,26.508136,51.571429,16.009973,18.730467,21.886077,8.773545,65.000000,79,A872801,6539


In [97]:
df_contagios_agg.sample()

,departamento,ano,semanas_epidemiologicas,cantidad_casos,poblacion,id_provincia,id_uta
11872,RIO PRIMERO,2020,17.0,1.0,53832,6,14105


In [99]:
merged.merge(
    df_contagios_agg,
    left_on=['id_departamento', 'anio', 'semana'],
    right_on=['id_uta', 'ano', 'semanas_epidemiologicas'],
    how='outer'

)

,id_estacion_x,anio,semana,fecha,precipitacion_pluviometrica,temperatura_minima,temperatura_maxima,temperatura_media,humedad_media,rocio_medio,...,id_estacion_y,id_interno,id_departamento,departamento,ano,semanas_epidemiologicas,cantidad_casos,poblacion,id_provincia,id_uta
0,NaN,NaN,<NA>,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,COMUNA 1,2018.0,16.0,1.0,254408.0,0.0,2001.0
1,NaN,NaN,<NA>,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,COMUNA 1,2018.0,17.0,1.0,254408.0,0.0,2001.0
2,NaN,NaN,<NA>,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,COMUNA 1,2018.0,18.0,1.0,254408.0,0.0,2001.0
3,NaN,NaN,<NA>,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,COMUNA 1,2019.0,7.0,2.0,255457.0,0.0,2001.0
4,NaN,NaN,<NA>,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,COMUNA 1,2019.0,49.0,1.0,255457.0,0.0,2001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28562,NaN,NaN,<NA>,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,USHUAIA,2024.0,18.0,5.0,84783.0,22.0,94014.0
28563,NaN,NaN,<NA>,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,USHUAIA,2024.0,19.0,1.0,84783.0,22.0,94014.0
28564,NaN,NaN,<NA>,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,USHUAIA,2024.0,21.0,2.0,84783.0,22.0,94014.0
28565,NaN,NaN,<NA>,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,USHUAIA,2025.0,19.0,1.0,86678.0,22.0,94014.0


## Clima

In [24]:
df_clima = pd.read_sql_query('SELECT * FROM clima', con=conn)
df_clima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112530 entries, 0 to 112529
Data columns (total 12 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   fecha                        112530 non-null  object 
 1   id_estacion                  112530 non-null  object 
 2   precipitacion_pluviometrica  112530 non-null  float64
 3   temperatura_minima           112530 non-null  float64
 4   temperatura_maxima           112530 non-null  float64
 5   temperatura_media            112530 non-null  float64
 6   humedad_media                112530 non-null  float64
 7   rocio_medio                  112530 non-null  float64
 8   tesion_vapor_media           112530 non-null  float64
 9   radiacion_global             112530 non-null  float64
 10  heliofania_efectiva          112530 non-null  float64
 11  heliofania_relativa          112530 non-null  float64
dtypes: float64(10), object(2)
memory usage: 10.3+ MB


In [26]:
df_clima.fecha = pd.to_datetime(df_clima.fecha)

In [29]:
df_clima.head()

,fecha,id_estacion,precipitacion_pluviometrica,temperatura_minima,temperatura_maxima,temperatura_media,humedad_media,rocio_medio,tesion_vapor_media,radiacion_global,heliofania_efectiva,heliofania_relativa
0,2018-01-01,A872801,0.000000,15.9,29.4,22.30069,52.0,12.40972,14.57236,27.55428,11.318030,79.0
1,2018-01-02,A872801,0.000000,14.1,32.2,23.74722,44.0,12.22327,14.48487,16.85220,5.099066,35.0
2,2018-01-03,A872801,0.000000,12.6,33.5,25.04375,38.0,10.18355,12.56902,18.67926,6.167767,43.0
3,2018-01-04,A872801,0.000000,17.1,36.4,27.37709,43.0,15.27765,17.47291,16.98126,5.186137,36.0
4,2018-01-05,A872801,0.589565,17.9,28.7,22.27430,91.0,17.58082,20.23586,2.17254,8.526174,59.5


In [30]:
df_clima['anio'] = df_clima.fecha.dt.year
df_clima['semana'] = df_clima.fecha.dt.isocalendar().week

In [34]:
columnas_groupby_clima = ['id_estacion', 'anio', 'semana']
df_clima_agg = df_clima.groupby(columnas_groupby_clima).mean().reset_index()

In [41]:
df_clima_agg.head(10)

,id_estacion,anio,semana,fecha,precipitacion_pluviometrica,temperatura_minima,temperatura_maxima,temperatura_media,humedad_media,rocio_medio,tesion_vapor_media,radiacion_global,heliofania_efectiva,heliofania_relativa
0,A872801,2018,1,2018-02-18 03:00:00,0.073696,14.925000,30.975000,23.480816,50.500000,12.928104,15.256275,20.920133,8.970463,62.562500
1,A872801,2018,2,2018-01-11 00:00:00,0.071282,20.857143,32.471429,26.344840,58.142857,17.314917,20.053073,19.498533,7.859546,55.142857
2,A872801,2018,3,2018-01-18 00:00:00,0.000000,17.800000,32.828571,25.153671,56.428571,16.678904,19.247406,23.093349,8.924465,63.428571
3,A872801,2018,4,2018-01-25 00:00:00,0.065213,18.557143,31.300000,24.316767,58.000000,16.353461,19.074726,22.105954,8.489425,61.142857
4,A872801,2018,5,2018-02-01 00:00:00,0.000000,19.814286,33.428571,26.380657,48.857143,16.066914,18.700631,24.891456,10.331836,75.285714
5,A872801,2018,6,2018-02-08 00:00:00,0.190602,20.000000,33.357143,26.508136,51.571429,16.009973,18.730467,21.886077,8.773545,65.000000
6,A872801,2018,7,2018-02-15 00:00:00,0.104044,16.542857,31.500000,23.878969,48.285714,13.606205,16.239880,24.580527,10.704467,80.428571
7,A872801,2018,8,2018-02-22 00:00:00,0.046151,16.657143,29.271429,22.894744,63.000000,15.696834,18.043853,19.724434,8.049689,61.714286
8,A872801,2018,9,2018-03-01 00:00:00,0.047689,18.814286,32.014286,25.124604,59.857143,17.483969,20.125569,20.344919,8.809178,68.857143
9,A872801,2018,10,2018-03-08 00:00:00,0.000000,15.128571,29.785714,22.371430,49.285714,11.884884,14.323001,20.147451,9.117978,72.714286


In [37]:
cols_numericas = df_clima_agg.drop(columns=['id_estacion', 'anio', 'semana', 'fecha']).columns.to_list()
cols_numericas

['precipitacion_pluviometrica',
 'temperatura_minima',
 'temperatura_maxima',
 'temperatura_media',
 'humedad_media',
 'rocio_medio',
 'tesion_vapor_media',
 'radiacion_global',
 'heliofania_efectiva',
 'heliofania_relativa']

In [42]:
# columnas climáticas a laggear
cols = [
    'precipitacion_pluviometrica',
    'temperatura_minima',
    'temperatura_maxima',
    'temperatura_media',
    'humedad_media',
    'rocio_medio',
    'tesion_vapor_media',
    'radiacion_global',
    'heliofania_efectiva',
    'heliofania_relativa'
]

# aseguramos orden temporal dentro de cada estación
df_clima_agg = df_clima_agg.sort_values(['id_estacion', 'fecha'])

# lags deseados (1, 2 y 3 semanas atrás)
lags = [1, 2, 3]

# construimos todas las columnas lag en bloque (rápido y claro)
lagged_blocks = []
g = df_clima_agg.groupby('id_estacion', sort=False)[cols]

for k in lags:
    block = g.shift(k)  # desplaza k filas dentro de cada estación
    block.columns = [f'{c}_lag{k}' for c in cols]
    lagged_blocks.append(block)

# concatenamos al df_clima_agg original
df_clima_agg_full = pd.concat([df_clima_agg] + lagged_blocks, axis=1)


In [44]:
df_clima_agg_full.shape

(16447, 44)

In [47]:
df_clima_agg_full = df_clima_agg_full.dropna().reset_index(drop=True)

In [58]:
df_clima_agg_full.head()

,id_estacion,anio,semana,fecha,precipitacion_pluviometrica,temperatura_minima,temperatura_maxima,temperatura_media,humedad_media,rocio_medio,...,precipitacion_pluviometrica_lag3,temperatura_minima_lag3,temperatura_maxima_lag3,temperatura_media_lag3,humedad_media_lag3,rocio_medio_lag3,tesion_vapor_media_lag3,radiacion_global_lag3,heliofania_efectiva_lag3,heliofania_relativa_lag3
0,A872801,2018,5,2018-02-01 00:00:00,0.000000,19.814286,33.428571,26.380657,48.857143,16.066914,...,0.071282,20.857143,32.471429,26.344840,58.142857,17.314917,20.053073,19.498533,7.859546,55.142857
1,A872801,2018,6,2018-02-08 00:00:00,0.190602,20.000000,33.357143,26.508136,51.571429,16.009973,...,0.000000,17.800000,32.828571,25.153671,56.428571,16.678904,19.247406,23.093349,8.924465,63.428571
2,A872801,2018,7,2018-02-15 00:00:00,0.104044,16.542857,31.500000,23.878969,48.285714,13.606205,...,0.065213,18.557143,31.300000,24.316767,58.000000,16.353461,19.074726,22.105954,8.489425,61.142857
3,A872801,2018,1,2018-02-18 03:00:00,0.073696,14.925000,30.975000,23.480816,50.500000,12.928104,...,0.000000,19.814286,33.428571,26.380657,48.857143,16.066914,18.700631,24.891456,10.331836,75.285714
4,A872801,2018,8,2018-02-22 00:00:00,0.046151,16.657143,29.271429,22.894744,63.000000,15.696834,...,0.190602,20.000000,33.357143,26.508136,51.571429,16.009973,18.730467,21.886077,8.773545,65.000000


In [59]:
estaciones = pd.read_sql_query('SELECT * FROM estaciones', con=conn)

estaciones.head()

,id_estacion,id_interno,nombre,latitud,longitud,id_provincia,id_localidad,id_departamento
0,413,A872872,25 DE MAYO - EEA PERGAMINO,-35.48,-60.13,0,0,6854
1,516,A872923,ABRA PAMPA - EEA ABRA PAMPA,-22.80,-65.83,9,78,38112
2,236,A872814,ALFONSO - EEA PERGAMINO,-33.91,-60.84,0,14,6623
3,393,A872885,ANDALGALA - EEA CATAMARCA,-27.61,-66.33,2,30,46119
4,403,A872912,ANDRESITO - EEA MONTECARLO,-25.62,-54.07,13,95,54049


In [70]:
estaciones.id_departamento = estaciones.id_departamento.astype(int) 

In [66]:
df_contagios_agg.sample(5)

,departamento,ano,semanas_epidemiologicas,cantidad_casos,poblacion,id_provincia,id_uta
1608,CAPAYAN,2025,14.0,1.0,19654,2,10042
5298,GENERAL ANGEL VERA PENALOZA,2025,20.0,1.0,3197,11,46056
8268,LAVALLE,2024,5.0,2.0,46428,12,50056
6932,IRIONDO,2020,8.0,1.0,72062,20,82056
12130,ROJAS,2024,14.0,17.0,24591,0,6686


In [75]:
df_contagios_agg.merge(estaciones[['id_departamento', 'id_estacion', 'id_interno']], left_on='id_uta', right_on='id_departamento')

,departamento,ano,semanas_epidemiologicas,cantidad_casos,poblacion,id_provincia,id_uta,id_departamento,id_estacion,id_interno
0,9 DE JULIO,2018,16.0,1.0,32942,20,82077,82077,515,A872962
1,9 DE JULIO,2018,17.0,1.0,32942,20,82077,82077,515,A872962
2,9 DE JULIO,2018,18.0,2.0,32942,20,82077,82077,515,A872962
3,9 DE JULIO,2018,19.0,4.0,32942,20,82077,82077,515,A872962
4,9 DE JULIO,2018,20.0,5.0,32942,20,82077,82077,515,A872962
...,...,...,...,...,...,...,...,...,...,...
4400,ZARATE,2024,17.0,3.0,132848,0,6882,6882,230,A872834
4401,ZARATE,2024,18.0,2.0,132848,0,6882,6882,230,A872834
4402,ZARATE,2024,19.0,2.0,132848,0,6882,6882,230,A872834
4403,ZARATE,2024,25.0,1.0,132848,0,6882,6882,230,A872834


In [76]:
departamentos = pd.read_sql_query('select * from departamentos', con=conn)
departamentos.sample(4)

,id_departamento,departamento,latitud,longitud,id_provincia,id_localidad
128,50021,GODOY CRUZ,-32.921528,-68.854923,12,87
47,86035,BANDA,-27.595488,-64.411170,21,141
143,54098,SAN IGNACIO,-27.251232,-55.530492,13,101
376,30091,TALA,-31.996204,-59.246491,7,71


In [73]:
df_contagios

,id_uta,departamento,provincia,ano,semanas_epidemiologicas,grupo_edad_id,grupo_edad_desc,cantidad_casos,poblacion,id_provincia
0,2003,COMUNA 3,BUENOS AIRES,2018,15.0,6,DE 15 A 19 ANOS,1.0,192945,0
1,2004,COMUNA 4,BUENOS AIRES,2018,6.0,6,DE 15 A 19 ANOS,1.0,239279,0
2,6091,BERAZATEGUI,BUENOS AIRES,2018,11.0,7,DE 20 A 24 ANOS,1.0,358262,0
3,6091,BERAZATEGUI,BUENOS AIRES,2018,14.0,9,DE 35 A 44 ANOS,2.0,358262,0
4,6091,BERAZATEGUI,BUENOS AIRES,2018,19.0,11,MAYORES DE 65 ANOS,1.0,358262,0
...,...,...,...,...,...,...,...,...,...,...
68121,90091,SIMOCA,TUCUMÁN,2025,19.0,11,MAYORES DE 65 ANOS,1.0,35572,23
68122,90091,SIMOCA,TUCUMÁN,2025,20.0,11,MAYORES DE 65 ANOS,1.0,35572,23
68123,90098,TAFI DEL VALLE,TUCUMÁN,2025,17.0,9,DE 35 A 44 ANOS,1.0,18511,23
68124,94007,RIO GRANDE,TIERRA DEL FUEGO,2025,21.0,10,DE 45 A 65 ANOS,1.0,107928,22
